## Notes/Ideas
In this Python notebook I will keep track of various things, including notes/documentation on how the old dBASE program works as well as explorations of ideas for the new software that I can play around with here before putting them in the real program.

In [1]:
# Libraries/Modules
import pandas as pd
import numpy as np
import os
import customtkinter as ctk
import functions as fn

# Static variables
data_dir = 'C:\\STMNU2\\Data'
dbf_dir = 'C:\\dbase\\gymtek'

### DBF Files

In [32]:
# Get a list of all the .dbf files that exist in the old program directory
dbf_files = [file for file in os.listdir(dbf_dir) if file.lower().endswith('.dbf')]
print(dbf_files)
# Need to ask Dad which of these are important. For now go ahead and convert everything to .csv
# so we can easily pull them into a Pandas dataframe and view them
for file in dbf_files:
    fn.dbf_to_csv(file)

['14toNov.dbf', 'BILLING.DBF', 'clsbymon.dbf', 'CLSBYMOX.DBF', 'clsbymxx.dbf', 'Copy (2) of clsbymon.dbf', 'Copy (2) of CLSBYMOX.DBF', 'Copy (2) of MNCHECK.DBF', 'Copy of clsbymon.dbf', 'Copy of CLSBYMOX.DBF', 'Copy of FIXMOX.DBF', 'Copy of MNCHECK.DBF', 'Copy of STUD00.dbf', 'Copy of STUD99.dbf', 'FINDBEN.dbf', 'MNCHECK.DBF', 'MNCHECK1.DBF', 'MONNEW.DBF', 'MONNEWS.DBF', 'NEWMARCH.DBF', 'STUD00 (1).dbf', 'STUD00.DBF', 'STUD00BA.DBF', 'STUD00BACK.DBF', 'STUD00withSarah.DBF', 'STUD1999.DBF', 'STUD2001.DBF', 'STUD2002.DBF', 'STUD2002B.DBF', 'STUD2002from2004.DBF', 'STUD2003.DBF', 'STUD2004.dbf', 'STUD2005.DBF', 'STUD2006.dbf', 'STUD2007.dbf', 'STUD2008.dbf', 'STUD2009.dbf', 'STUD2010.dbf', 'STUD2011.dbf', 'STUD2012.DBF', 'STUD2013.dbf', 'STUD2014.dbf', 'STUD2015.dbf', 'STUD2016.dbf', 'STUD2017.dbf', 'STUD2018.DBF', 'STUD2019 (2).dbf', 'STUD2019.dbf', 'STUD2020.dbf', 'STUD2021.dbf', 'STUD2022.DBF', 'STUD6MO.DBF', 'STUD99.DBF', 'STUD99.DBF.dbf', 'STUD99back.dbf', 'STUD99O.dbf', 'STUD99ol.db

In [ ]:
students.sort_values(by=['LNAME','FNAME']).index.get_loc(791)
students.dropna(subset=['FNAME','LNAME']).shape[0] - 1


5922

: 

#### F1: Student Info
Replicating the functionality of the 'student info' feature in the original program (press F1 on main menu)

In [2]:
# Find out which 'STUD' dbf has the most records
csv_files = [file for file in os.listdir(data_dir) if 'stud' in file.lower()]
record_counts = []
for file in csv_files:
    record_counts.append(pd.read_csv(os.path.join(data_dir, file)).shape[0])

most_records = csv_files[np.array(record_counts).argmax()]
print(most_records)
students = pd.read_csv(os.path.join(data_dir, most_records))
display(students)

STUD00.csv


,FNAME,MIDDLE,LNAME,SEX,BIRTHDAY,CLASS,LEVEL,ADDRESS,CITY,STATE,...,OCTPAY,OCTDATE,OCTBILL,NOVPAY,NOVDATE,NOVBILL,DECPAY,DECDATE,DECBILL,REGBILL
0,AUDRA,A.,BILBREY,F,1982-02-13,L6,0,3257 OLD EAGLE LAKE RD.,WINTER HAVEN,FL,...,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
1,JACKIE,NaN,MARTIN,F,1982-12-29,TB,0,1942 19TH ST N.W.,WINTER HAVEN,FL,...,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
2,KATIE,NaN,MCINTIRE,F,1979-12-05,GA,0,6632 W. NEWMAN CIRCLE,LAKELAND,FL,...,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
3,CAROLYN,NaN,CABANAS,F,NaN,L6,0,3305 HUGHES ST.,LAKELAND,FL,...,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
4,ADAM,J,ROBINSON,M,1978-06-12,GB,0,86 JOHN'S RD.,WAUCHULA,FL,...,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5997,AXEL,NaN,HANSEN,NaN,2029-03-25,PS,0,1334 TURKEY TRL,LAKELAND,FL,...,23.0,2024-10-21,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
5998,LAUREL,NaN,LANG,NaN,2015-10-14,GB,0,4910 HERNDON WAY,AUBURNDALE,FL,...,25.0,2024-10-21,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
5999,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,...,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
6000,JOANNA,NaN,WITTE,NaN,2019-04-14,PS,0,4665 SWINDELL RD,LAKELAND,FL,...,75.0,2024-10-22,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN


In [25]:
# 'lname' will represent the user input, case-insensitive
lname = ctk.CTkInputDialog(text="Last Name:", title="Student Info").get_input().upper()

# Find all students with a match, and then select the first record alphabetically 
first_match = students[students['LNAME'].str.upper().str.startswith(lname, na=False)
                ].sort_values(by=['LNAME','FNAME']
                ).head(1)
pd.set_option('display.max_columns', None)
display(first_match)


,FNAME,MIDDLE,LNAME,SEX,BIRTHDAY,CLASS,LEVEL,ADDRESS,CITY,STATE,ZIP,PHONE,NOTE1,NOTE2,NOTE3,INSTRUCTOR,DAYTIME,INST2,DAYTIME2,INST3,DAYTIME3,REGFEE,REGFEEDATE,EMAIL,STATUS,ENROLLDATE,STUDENTNO,MOMNAME,DADNAME,MONTHLYFEE,BALANCE,JANPAY,JANDATE,JANBILL,FEBPAY,FEBDATE,FEBBILL,MARPAY,MARDATE,MARBILL,APRPAY,APRDATE,APRBILL,MAYPAY,MAYDATE,MAYBILL,JUNPAY,JUNDATE,JUNBILL,JULPAY,JULDATE,JULBILL,AUGPAY,AUGDATE,AUGBILL,SEPPAY,SEPDATE,SEPBILL,OCTPAY,OCTDATE,OCTBILL,NOVPAY,NOVDATE,NOVBILL,DECPAY,DECDATE,DECBILL,REGBILL
790,ELIZABETH,NaN,ZUCKER,NaN,1998-07-29,GB,0,PO BOX 3537,PLANT CITY,FL,33563,H/813-659-2605 DC/813-763-5853,TAKING SUMMER09 OFF AB,LM 8/24 TO SEE IF COMING,BACK KO,AMANDA,M 5:05,NaN,NaN,NaN,NaN,21.0,2008-11-04,NaN,False,2008-04-15,1791,MARIA C/813-624-5241,STEVEN,75.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN


In [69]:
student_info = students.iloc[1110]
student_info = student_info.astype('string').str.title()
student_info['MONTHLYFEE']

'65.0'

#### Classes

In [6]:
classes = pd.read_csv(data_dir + '\\clsbymon.csv')
display(classes.columns)

Index(['SESSION', 'TEACH', 'TEACH2', 'DAYOFWEEK', 'TIMEOFDAY', 'CLASSTIME',
       'SECONDTIME', 'CODE', 'CLASSNAME', 'MAX',
       ...
       'TRIAL7', 'T7PHONE', 'T7DATE', 'TRIAL8', 'T8PHONE', 'T8DATE', 'NOTE1',
       'NOTE2', 'NOTE3', 'NOTE4'],
      dtype='object', length=111)